In [31]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from math import sqrt

In [32]:
def make_category_1(file_path):


    f = open(file_path).read()
    data = json.loads(f)
    playlists = data["playlists"]
    df = pd.DataFrame(playlists)
    train, test = train_test_split(df, test_size=0.1)

    r_test_data = {'pid' : list(test['pid']), 'name' : list(test['name']), 'num_holdouts' : list(test['num_tracks']), 'tracks' : [{} for _ in range(test.shape[0])], 'num_samples' : [0 for _ in range(test.shape[0])], 'num_tracks' : test['num_tracks']}
    r_test = pd.DataFrame(r_test_data)
    
    train = train.T.to_dict().values()
    test = test.T.to_dict().values()
    r_test = r_test.T.to_dict().values()
    
    with open('./train.json', 'w+') as f:
        f.write(json.dumps(train))
    with open('./test_R.json', 'w+') as f:
        f.write(json.dumps(test))
    with open('./test.json', 'w+') as f:
        f.write(json.dumps(r_test))
        
        
make_category_1('mpd.slice.0-999.json')

<h1> train_set </h1>

In [33]:
f = open('./train.json').read()
data = json.loads(f)
train_df = pd.DataFrame(data)

<h1> test_set </h1>

In [34]:
f = open('./test_R.json').read()
data = json.loads(f)
test_df = pd.DataFrame(data)

<h1> train_set_name & test_set_name </h1>

In [35]:
train_playlist_name = list(set(train_df['name']))
test_playlist_name = list(set(test_df['name']))

In [36]:
train_playlist_name = [train_playlist_name[i:i + 10] 
                       for i in xrange(0, len(train_playlist_name), 10)]
train_playlist_name

[[u'summer country',
  u'This Is What You Came For',
  u'Pregame',
  u'dance',
  u'Relaxing Music',
  u'ale',
  u'ouch',
  u'fuego',
  u'Say My Name',
  u'sleep'],
 [u'lit songs ',
  u'The Mix',
  u'taylor swift',
  u'mellow',
  u'classic rock',
  u'Sleep Music',
  u'twenty sixteen',
  u'R&B',
  u' Frozen',
  u'guitar.'],
 [u'Sweat.It.Out.',
  u'Dinner Party',
  u'wake up',
  u'\U0001f608\U0001f608\U0001f608',
  u'tb',
  u'sound',
  u'Feel Good',
  u'Greatest Hits',
  u'bang bang. ',
  u'EDC'],
 [u'Meow',
  u'Summer Playlist',
  u'Top hits',
  u'Throwback',
  u'Stuff',
  u'Sugar',
  u'TRIPPIn',
  u'ROCK\U0001f3b8',
  u'Rock en espa\xf1ol ',
  u"Don't"],
 [u'\U0001f62d\U0001f62d',
  u'Summer drives',
  u'Mixtape',
  u'vibes',
  u'sb2k17',
  u'Road Trip Mix',
  u'Wedding Songs',
  u'Get pumped ',
  u'Panda',
  u'Kevin'],
 [u'PlayStation',
  u'Playlist',
  u'good good',
  u'Tegan and Sara',
  u'new stuff',
  u'running ',
  u'Lit af',
  u'90s',
  u'my heart',
  u'issa playlist'],
 [u'The D

In [37]:
len(train_playlist_name[0])

10

In [38]:
train_playlist_name[0].extend(train_playlist_name[1])
len(train_playlist_name[0])

20

<h1> cross validation test </h1>

In [39]:
train_playlist_name = list(set(train_df['name']))

train_playlist_name = [train_playlist_name[i:i + len(train_playlist_name)/10] 
                       for i in xrange(0, len(train_playlist_name), len(train_playlist_name)/10)]
topic_num = 7
max = 0
max_topic_num = 0
for idx in range(0, len(train_playlist_name)):
    #0~9 까지 validation
    validation_texts = [[word for word in name.lower().split()]for name in train_playlist_name[idx]]
    
    #train_set after slice
    train_texts = []
    for tidx in range(0,len(train_playlist_name)):
        if(tidx == idx):
            continue
        train_texts.extend(train_playlist_name[tidx])
    train_texts = [[word for word in name.lower().split()]
                  for name in train_texts]
    
    #LDA model create
    dictionary = corpora.Dictionary(train_texts)
    corpus = [dictionary.doc2bow(text) for text in train_texts]
    model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
                                    num_topics = topic_num, random_state = 0,
                                    chunksize = 10000, passes = 10)
    
    #document 별 topic 다항분포 train
    train_dic = {}
    for idx in range(0, len(train_texts)):
        t = train_texts[idx]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        train_dic[name] = dict(get_document_topics)
    train_dic = pd.DataFrame(train_dic)
    #print train_dic
    
    #document 별 topic 다항분포 validation
    validation_dic = {}
    for idx in range(0, len(validation_texts)):
        t = validation_texts[idx]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        validation_dic[name] = dict(get_document_topics)
    validation_dic =pd.DataFrame(validation_dic).T
    #print validation_dic
    
    result = []
    for trainName in train_dic.columns:
        idx = 0
        c = 0
        columns_ones = []
        for valName in validation_dic.index:
            for idx in range(0, topic_num):
                c += pow(train_dic[trainName][idx] - validation_dic[idx][valName], 2)
            c = 1/(1+sqrt(c))
            columns_ones.append(c)
        result.append(columns_ones)
    #train_playlist X validation_playlist_name 유클리디안 거리 행렬 생성    
    result = pd.DataFrame(result, columns = validation_dic.index, index = train_dic.columns)
    
    #validation_Set evaluation
    count = 0
    c = 0;
    for name in result.columns:
        #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
        result_playlist_name = result[name].sort_values(ascending = False).head(10).index
        #print count
        count+=1
    
        #test_playlist 생성
        validation_playlist = []
        for idx in range(0, len(train_df['name'])):
            if(train_df['name'][idx].lower() == name):
                val_track_uri = pd.DataFrame(train_df['tracks'][idx])
                val_playlist = list(val_track_uri['track_uri'])
        #print len(test_playlist)        
        if(len(val_playlist) == 0):
            continue
            
        #비교할 train_playlist 생성
        train_playlist = []
        for train_name in result_playlist_name:
            for idx in range(0,len(train_df['name'])):
                if(train_df['name'][idx].lower() == train_name):
                    df = pd.DataFrame(train_df['tracks'][idx])
                    train_playlist += list(df['track_uri'])
                    if(len(train_playlist) >= 500):
                        continue
                    elif(len(train_playlist) + len(list(df['track_uri'])) >= 500):
                        temp = list(df['track_uri'])
                        train_playlist += temp[:-(len(train_playlist)-500)]
                    else:
                        train_playlist += list(df['track_uri'])
        #print len(train_playlist)      
        #r-precision 계산
        c += 1.0 *len(set(train_playlist).intersection(set(val_playlist)))/len(set(val_playlist)) *100
    
    #avarage result
    c = 1.0 * c/(count)
    print ("average : {}, topic_num : {}".format(c, topic_num))
    if max < c:
        max = c
        max_topic_num = topic_num
        maxModel = model

    topic_num += 2 #3,5,7,9,11,13,15,....
print ("max_average : {}, max_topic_num : {}".format(max, max_topic_num)) 

average : 13.7393619183, topic_num : 7
average : 12.0374625843, topic_num : 9
average : 12.8472121793, topic_num : 11
average : 11.6889592695, topic_num : 13
average : 9.65246448546, topic_num : 15
average : 15.4721993869, topic_num : 17
average : 17.9346060637, topic_num : 19
average : 23.835930924, topic_num : 21
average : 12.3590698144, topic_num : 23
average : 4.12835764599, topic_num : 25
average : 1.85185185185, topic_num : 27
max_average : 23.835930924, max_topic_num : 21


<h1> test </h1>

In [30]:
train_playlist_name = list(set(train_df['name']))
test_playlist_name = list(set(test_df['name']))

#train_playlist_name
#소문자로
train_texts = [[word for word in name.lower().split()]
         for name in train_playlist_name]

#test_playlist_name
#역시 소문자로
test_texts = [[word for word in name.lower().split()]
         for name in test_playlist_name]

topic_num =27
 #LDA model create
#dictionary = corpora.Dictionary(train_texts)
#corpus = [dictionary.doc2bow(text) for text in train_texts]
#model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
#                                   num_topics = topic_num, random_state = 0,
#                                    chunksize = 20000, passes = 10)
model = maxModel   
    #document 별 topic 다항분포 train

train_dic = {}
for idx in range(0, len(train_texts)):
    t = train_texts[idx]
    bow = dictionary.doc2bow(t)
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    train_dic[name] = dict(get_document_topics)
train_dic = pd.DataFrame(train_dic)
#print train_dic
    
    #document 별 topic 다항분포 validation
test_dic = {}
for idx in range(0, len(test_texts)):
    t = test_texts[idx]
    bow = dictionary.doc2bow(t)
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    test_dic[name] = dict(get_document_topics)
test_dic =pd.DataFrame(test_dic).T
#print validation_dic

result = []
for trainName in train_dic.columns:
    idx = 0
    c = 0
    columns_ones = []
    for testName in test_dic.index:
        for idx in range(0, topic_num):
            c += pow(train_dic[trainName][idx] - test_dic[idx][testName], 2)
        c = 1/(1+sqrt(c))
        columns_ones.append(c)
    result.append(columns_ones)
#train_playlist X validation_playlist_name 유클리디안 거리 행렬 생성    
result = pd.DataFrame(result, columns = test_dic.index, index = train_dic.columns)
#print result   
    #validation_Set evaluation
count = 0
c = 0;
for name in result.columns:
    #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
    result_playlist_name = result[name].sort_values(ascending = False).head(10).index
    print count
    
    #test_playlist 생성
    test_playlist = []
    for idx in range(0, len(test_df['name'])):
        if(test_df['name'][idx].lower() == name):
            test_track_uri = pd.DataFrame(test_df['tracks'][idx])
            test_playlist = list(test_track_uri['track_uri'])
    print "test_playlist length :{}".format(len(test_playlist))        
    if(len(test_playlist) == 0):
        continue
            
        #비교할 train_playlist 생성
    train_playlist = []
    for train_name in result_playlist_name:
        for idx in range(0,len(train_df['name'])):
            if(train_df['name'][idx].lower() == train_name):
                df = pd.DataFrame(train_df['tracks'][idx])
                temp =[]
                if(len(train_playlist) >= 500):
                    continue
                elif(len(train_playlist) + len(list(df['track_uri'])) >= 500):
                    temp = list(df['track_uri'])
                    train_playlist += temp[:-(len(train_playlist)-500)]
                else:
                    train_playlist += list(df['track_uri'])
    print "total_train_playlist : {}".format(len(train_playlist))      
    #r-precision 계산
    #if(len(set(train_playlist).intersection(set(test_playlist))) == 0):
    #    continue
    c += 1.0 *len(set(train_playlist).intersection(set(test_playlist)))/len(set(test_playlist)) *100
    count+=1
    print "r-precision summation: {}".format(c)
    print "count : {}".format(count)
    #avarage result
c = 1.0 * c/(count)
print "total average : {}".format(c)

0
test_playlist length :22
total_train_playlist : 500
r-precision summation: 0.0
count : 1
1
test_playlist length :0
1
test_playlist length :53
total_train_playlist : 500
r-precision summation: 3.84615384615
count : 2
2
test_playlist length :57
total_train_playlist : 500
r-precision summation: 7.35492577598
count : 3
3
test_playlist length :103
total_train_playlist : 500
r-precision summation: 17.0636636401
count : 4
4
test_playlist length :38
total_train_playlist : 500
r-precision summation: 17.0636636401
count : 5
5
test_playlist length :10
total_train_playlist : 500
r-precision summation: 17.0636636401
count : 6
6
test_playlist length :18
total_train_playlist : 500
r-precision summation: 33.7303303067
count : 7
7
test_playlist length :44
total_train_playlist : 500
r-precision summation: 33.7303303067
count : 8
8
test_playlist length :0
8
test_playlist length :209
total_train_playlist : 500
r-precision summation: 34.6965138816
count : 9
9
test_playlist length :47
total_train_playlist

count : 80
80
test_playlist length :49
total_train_playlist : 500
r-precision summation: 310.028889238
count : 81
81
test_playlist length :80
total_train_playlist : 500
r-precision summation: 312.528889238
count : 82
82
test_playlist length :23
total_train_playlist : 500
r-precision summation: 316.876715325
count : 83
83
test_playlist length :25
total_train_playlist : 500
r-precision summation: 336.876715325
count : 84
84
test_playlist length :47
total_train_playlist : 500
r-precision summation: 341.132034474
count : 85
85
test_playlist length :50
total_train_playlist : 500
r-precision summation: 341.132034474
count : 86
86
test_playlist length :115
total_train_playlist : 500
r-precision summation: 346.441768987
count : 87
87
test_playlist length :191
total_train_playlist : 500
r-precision summation: 348.536009825
count : 88
total average : 3.96063647529
